In [ ]:
import os
os.environ['DISABLE_V2_BEHAVIOR'] = '1'

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
import ktrain
from ktrain import text as txt

In [ ]:
# load data
(trn, val, preproc) = txt.entities_from_txt('DATA_ANNOTATED.csv',
                                            sentence_column='Sentence',
                                            word_column='Word',
                                            tag_column='Tag', 
                                            data_format='gmb',
                                            use_char=True)

In [ ]:
model = txt.sequence_tagger('bilstm-crf', preproc)

In [ ]:
learner = ktrain.get_learner(model, train_data=trn, val_data=val)

In [ ]:
history = learner.fit(lr=1e-3, n_cycles=2, cycle_len=3) 

In [ ]:
learner.validate(class_names=preproc.get_classes())

### Prevendo novas frases

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.predict('As of 2019, Donald Trump is still the President of the United States.')